In [1]:
!pip install -U langchain openai chromadb langchainhub bs4

  Using cached langchain-0.0.351-py3-none-any.whl.metadata (13 kB)
  Using cached openai-1.5.0-py3-none-any.whl.metadata (17 kB)
  Using cached chromadb-0.4.20-py3-none-any.whl.metadata (7.3 kB)
  Using cached langchainhub-0.1.14-py3-none-any.whl.metadata (478 bytes)
  Using cached bs4-0.0.1-py3-none-any.whl
  Using cached langsmith-0.0.72-py3-none-any.whl.metadata (10 kB)
  Using cached distro-1.8.0-py3-none-any.whl (20 kB)
  Using cached posthog-3.1.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached onnxruntime-1.16.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.3 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.42b0-py3-none-any.whl.metadata (2.3 kB)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached importlib_resources-6.1.1-py3-none-any.whl.metadata (4.1 kB)
  Using cached typer-0.9.0-py3-none-any.whl (45 kB)
  Using cached kubernetes-28.1.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached types_requests-2.31.0.10-py3-non

In [2]:
!pip install sentence-transformers

  Using cached sentence_transformers-2.2.2-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 7.3 MB/s eta 0:00:00
  Using cached scikit_learn-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached scipy-1.11.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_6

In [6]:
!pip install opensearch-py
!pip install requests-aws4auth
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.6/258.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 70.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.1 MB/s eta 0:00:00


In [1]:
import bs4
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings

from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate

from opensearchpy import OpenSearch, Urllib3HttpConnection, Urllib3AWSV4SignerAuth
from langchain.vectorstores import OpenSearchVectorSearch
from requests_aws4auth import AWS4Auth
import boto3

In [2]:
# Constants, variables
index_suffix = "_20240101"
es_embeddings = HuggingFaceEmbeddings(model_name="distiluse-base-multilingual-cased-v1")
#es_embeddings = HuggingFaceEmbeddings(model_name="distiluse-base-multilingual-cased-v2")
#es_embeddings = HuggingFaceEmbeddings(model_name="paraphrase-multilingual-MiniLM-L12-v2")
#es_embeddings = HuggingFaceEmbeddings(model_name="paraphrase-multilingual-mpnet-base-v2")
#es_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
#es_embeddings = OpenAIEmbeddings()

In [3]:
# Check Connection
es_host = 'vpc-dev-search-es-250-crdyurhlzxphmrjdfmtz7m23nu.ap-northeast-2.es.amazonaws.com'
es_url = 'https://' + es_host
region = 'ap-northeast-2'

session = boto3.Session()
credentials = session.get_credentials()
awsauth = Urllib3AWSV4SignerAuth(credentials, region, 'es')

es = OpenSearch(
    hosts = [{'host': es_host, 'port': 443}],
    http_auth = awsauth,
    use_ssl = True,
    verify_certs = True,
    connection_class = Urllib3HttpConnection,
    pool_maxsize = 20
)
print(es.info())

{'name': '07d3e7c3ade78c4a51db185ae66c04b6', 'cluster_name': '175979101058:dev-search-es-250', 'cluster_uuid': 'uPQyc5N1Ri-lw1OxbsjxaQ', 'version': {'number': '7.10.2', 'build_type': 'tar', 'build_hash': 'unknown', 'build_date': '2023-04-20T10:37:24.437846Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.10.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'The OpenSearch Project: https://opensearch.org/'}


In [4]:
# Load documents from wikipedia
loader = WebBaseLoader(
    web_paths=("https://en.wikipedia.org/wiki/SpaceX_Starship_integrated_flight_test_1",
               "https://en.wikipedia.org/wiki/2023_United_States_banking_crisis",
               "https://en.wikipedia.org/wiki/2023_Israel%E2%80%93Hamas_war",
               "https://en.wikipedia.org/wiki/2023_Hawaii_wildfires",
               "https://en.wikipedia.org/wiki/Gemini_(language_model)"
              ),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("mw-body")
#            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [5]:
print(len(docs))

5


In [6]:
for doc in docs:
    print(len(doc.page_content))

40785
73264
338334
95569
31014


In [7]:
print(docs[0].page_content)







Toggle the table of contents







SpaceX Starship integrated flight test 1



11 languages




العربيةČeštinaفارسیFrançaisBahasa MelayuپنجابیPortuguêsไทยУкраїнськаاردو中文

Edit links











ArticleTalk





English

















ReadEditView history







Tools





Tools
move to sidebar
hide



		Actions
	


ReadEditView history





		General
	


What links hereRelated changesUpload fileSpecial pagesPermanent linkPage informationCite this pageGet shortened URLWikidata item





		Print/export
	


Download as PDFPrintable version





		In other projects
	


Wikimedia Commons

























From Wikipedia, the free encyclopedia


First integrated test launch of SpaceX Starship
This article is about the first attempt at near-orbital flight. For other flights, see SpaceX Starship flight tests.


Starship's first integrated flight testFully stacked Starship vehicle during its first flightMission typeFlight testOperatorSpaceXMission duration3 minutes, 57 seconds (

In [31]:
print(len(splits))

829


In [38]:
print(splits[104])

page_content='By March 16, large inter-bank flows of funds were occurring to shore up bank balance sheets and numerous analysts were reporting on a more general U.S. banking crisis. Many banks had invested their reserves in U.S. Treasury securities, which had been paying low interest rates. As the Federal Reserve began raising rates in 2022, bond prices declined decreasing the market value of bank capital reserves, leading some banks to sell the bonds at steep losses as yields on new bonds were much higher.[7]\nOn March 17, President Joe Biden stated that the banking crisis had calmed down,[111] while the New York Times said that the March banking crisis was hanging over the economy and had rekindled fear of recession as business borrowing would become more difficult as many regional and community banks would have to reduce lending.[112][113]' metadata={'source': 'https://en.wikipedia.org/wiki/2023_United_States_banking_crisis'}


In [40]:
print(splits[105])

page_content='Late on Sunday, the Federal Reserve and several other central banks announced significant USD liquidity measures in order to calm market turmoil.[114] In a "coordinated action to enhance the provision of liquidity through the standing U.S. dollar swap line arrangements", the U.S. Federal Reserve, the Bank of Canada, Bank of Japan, European Central Bank, and Swiss National Bank joined together to organize daily U.S. dollar swap operations. These swaps had previously been set up to occur on a weekly cadence.[115]\nThe share price of PacWest had fallen sharply on 3 May after the bank announced that it was \'considering strategic options including a sale\'. On 4 May share trading was suspended as the sell-off marked a further 42% loss with other US regional banks, including First Horizon, Metropolitan Bank and Western Alliance, also being affected.[116][117]' metadata={'source': 'https://en.wikipedia.org/wiki/2023_United_States_banking_crisis'}


In [8]:
# Functions
settings = {
  "settings": {
    "number_of_shards": 3,
    "number_of_replicas": 0,
    "index.search.slowlog.threshold.query.warn": "3s",
    "knn.algo_param": {
      "ef_search": "512"
    },
    "knn": "true"
  },
  "mappings": {
    "properties": {
      "vector_field": {
          "type": "knn_vector",
          "dimension": 384,
          "method": {
            "engine": "nmslib",
            "space_type": "l2",
            "name": "hnsw",
            "parameters": {
              "ef_construction": 512,
              "m": 16
            }
          }
      }
    }
  }
}

def make_index(es, index_name):
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print("delete index :", index_name)
    es.indices.create(index=index_name, body=settings)
    print("create index :", index_name)
    
def switch_alais(es, alais_name, new_index_name):
    indices = list()
    if es.indices.exists_alias(name=alais_name):
        indices=es.indices.get_alias(alais_name).keys()
    
    update_alias = list()
    for index in indices:
        update_alias.append({"remove": {"alias": alais_name, "index": index}})
    update_alias.append({"add": {"alias": alais_name, "index": new_index_name}})
    action = { "actions": update_alias }

    if update_alias:
        # repoint the alias to point to the newly created index
        result = es.indices.update_aliases(
            body={
                **action
            }
        )
        print(result)

In [9]:
# global variable - es_embeddings, es_url, awsauth
def indexing_dump(index_name, docs, make_index_yn=True, mapping=None):
    if make_index_yn:
        make_index(es, index_name)
    
    if mapping is not None:
        resp = es.indices.put_mapping(index=index_name, body={**mapping}, doc_type="_doc")
        print(resp)

    vectorstore = OpenSearchVectorSearch.from_documents(
        docs,
        es_embeddings,
        opensearch_url=es_url,
        http_auth=awsauth,
        timeout=300,
        use_ssl=True,
        verify_certs=True,
        connection_class=Urllib3HttpConnection,
        index_name=index_name
    )

In [10]:
# Index documents
index_alias_wikipedia = 'wikipedia_en_cased_v1'

index_name_wikipedia = index_alias_wikipedia + index_suffix

In [ ]:
es.indices.delete(index=index_name_wikipedia)

In [11]:
# make_index_yn=False -> index를 OpenSearchVectorSearch.from_documents() 호출시 생성하도록 함
indexing_dump(index_name_wikipedia, docs, make_index_yn=False)

In [12]:
switch_alais(es, index_alias_wikipedia, index_name_wikipedia)

{'acknowledged': True}


In [13]:
# Search documents
def similarity_search(index_name, query):
    vectorstore = OpenSearchVectorSearch(
        opensearch_url=es_url,
        embedding_function=es_embeddings,
        http_auth=awsauth,
        timeout=3000,
        use_ssl=True,
        verify_certs=True,
        connection_class=Urllib3HttpConnection,
        index_name=index_name,
    )

    results = vectorstore.similarity_search_with_score(
        query,
    #    search_type="script_scoring",
    #    space_type="cosinesimil",
    #    vector_field="message_embedding",
    #    text_field="message",
    #    metadata_field="message_metadata",
        k=4
    )

    # for doc in docs:
    #     print(doc.page_content)
    return results

In [30]:
results = similarity_search(index_name_wikipedia, "why silicon vally bank bankrupt?")
for doc in results:
    print("------")
    print(doc[1])
    print(doc[0].page_content[:100])

------
0.40918428






Toggle the table of contents







2023 United States banking crisis



14 languages




العر
------
0.39445734






Toggle the table of contents







Gemini (language model)



18 languages




العربيةAzərbay
------
0.3924299






Toggle the table of contents







SpaceX Starship integrated flight test 1



11 languages


------
0.3784615






Toggle the table of contents







2023 Hawaii wildfires



32 languages




العربيةБеларуска


In [29]:
print(docs[1].page_content[3600:3900])

 billion capital infusion from a group of major banks in March, FRB continued to destabilize and its stock price plummeted as the FDIC prepared to take it into receivership and find a buyer on April 29.[9][10] On May 1, the FDIC announced that First Republic had been closed and sold to JPMorgan Chas


In [28]:
results = similarity_search(index_name_wikipedia, docs[1].page_content[3600:3900])
for doc in results:
    print("------")
    print(doc[1])
    print(doc[0].page_content[:100])

------
0.3954387






Toggle the table of contents







SpaceX Starship integrated flight test 1



11 languages


------
0.39058986






Toggle the table of contents







2023 United States banking crisis



14 languages




العر
------
0.38361618






Toggle the table of contents







2023 Israel–Hamas war



77 languages




Afrikaansالعربية
------
0.3782477






Toggle the table of contents







Gemini (language model)



18 languages




العربيةAzərbay


In [12]:
results = similarity_search(index_name_wikipedia, "잼버리")
for doc in results:
    print("------")
    print(doc[1])
    print(doc[0].page_content[:100])

------
0.4690492






목차 토글







2023년 튀르키예-시리아 지진



77개 언어




Afrikaansالعربيةالدارجةঅসমীয়াAzərbaycancaتۆرکجهБ
------
0.45454687






목차 토글







제25회 세계 스카우트 잼버리



13개 언어




العربيةবাংলাČeštinaEnglishEspañolFrançaisעבריתItal
------
0.4518326






목차 토글







2023년 이스라엘-하마스 전쟁



78개 언어




AfrikaansالعربيةالدارجةمصرىAzərbaycancaБеларуская
------
0.4474291






목차 토글







SVB 파산 사건



16개 언어




AfrikaansالعربيةБеларускаяΕλληνικάEnglishEspañolفارسیFran
